In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings

In [ ]:

# CFM 101 - Group Assignment 2025
# Robo-Advising Challenge
# Competition Goal: Market Beat - Highest return above the benchmark average

warnings.filterwarnings('ignore')


# ============================================================================
# CONFIGURATION
# ============================================================================

INITIAL_CAPITAL = 1_000_000  # CAD
USD_TO_CAD = 1.41  # Exchange rate (update as needed)
VOLUME_CHECK_START = '2023-10-01'
VOLUME_CHECK_END = '2024-09-30'
MIN_STOCKS = 10
MAX_STOCKS = 25
MAX_WEIGHT = 0.15
MAX_SECTOR_WEIGHT = 0.40
MIN_TRADING_DAYS = 18

TRAINING_DAY_BEGIN = "2022-01-01"
TRAINING_DAY_END = "2024-01-02"
DAY_AFTER_TRAINING_DAY_END = "2024-01-03"
YEAR_AFTER_TRAINING_DAY_END = "2025-01-03"
RISK_FREE_RATE = 2/12

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

us_ticker_ex = yf.Ticker("AAPL")
cad_ticker_ex = yf.Ticker("Shop.to")

def first_trading_days_of_each_month (start_date, end_date):
    us_ticker_ex_hist = us_ticker_ex.history(start= start_date, end = end_date)
    us_ticker_ex_hist.index = us_ticker_ex_hist.index.date
    cad_ticker_ex_hist = cad_ticker_ex.history(start= start_date, end = end_date)
    cad_ticker_ex_hist.index = cad_ticker_ex_hist.index.date

   
    valid_dates = np.intersect1d(us_ticker_ex_hist.index, cad_ticker_ex_hist.index)
    
    first_trading_days= []
    prev_month = 0
    for date in valid_dates:
        if date.month != prev_month:
            first_trading_days.append(date)
            prev_month = date.month

    return first_trading_days


def create_df(stocks, dates, distribution):
    start_date = dates[0]
    end_date = dates[len(dates)-1]+pd.Timedelta(days=1)
    
    # creating a dataframe for each ticker with its close price at every first trading day of the month
    stocks_df = []

    
    for ticker in stocks:
        temp_df = ticker.history(start=start_date, end=end_date)  
        temp_df.index = temp_df.index.date
        count = 0

        df_dict = {'Date': dates, 
               'Close': []}

        for i in range (len (temp_df)):
            if temp_df.index[i] == dates[count]:
                close = temp_df['Close'].iloc[i]
                df_dict['Close'].append(close)
                count += 1


        stock_df = pd.DataFrame(df_dict)
        stock_df = stock_df.set_index("Date")
      
        stocks_df.append(stock_df)

    # finding out how many shares in each stock will be bought
    shares_in_stocks = []

    money_in_each_stock = []
    for i in range (len(stocks)):
        money_in_each_stock.append(distribution[i]*INITIAL_CAPITAL)
    
    for i in range (len(stocks)):
        close = stocks_df[i]['Close'].iloc[0]
        shares_in_stocks.append(shares(stocks[i], close, money_in_each_stock[i]))

    # creating a dataframe that stores the portfolio value at every first trading day of the month
    stock_portfolio = {
        "Date": dates,
        "Portfolio Value": []}
    
    for i in range (len(dates)):
        portfolio_value = 0
        for j in range(len(stocks)):
            close = stocks_df[j]['Close'].iloc[i]
            if stocks[j].info['country'] != 'Canada':
                close *= USD_TO_CAD
            portfolio_value += close*shares_in_stocks[j]
            
        stock_portfolio["Portfolio Value"].append(portfolio_value)
    
    stock_portfolio = pd.DataFrame(stock_portfolio)
    stock_portfolio = stock_portfolio.set_index("Date")
    stock_portfolio['Percentage Returns']= stock_portfolio['Portfolio Value'].pct_change()*100
    stock_portfolio.drop(index=stock_portfolio.index[0], inplace = True)
    return stock_portfolio

def shares(ticker, price, money):
    cad_transaction_fee_per_stock = 0.01*USD_TO_CAD
    cad_max_transaction_fee = 2.15*USD_TO_CAD
    cad_price = price

    if ticker.info['country'] != 'Canada':
        cad_price = price*USD_TO_CAD

    return max(money/(cad_price+cad_transaction_fee_per_stock),
               (money-cad_max_transaction_fee)/cad_price)

    

def calculate_fee(shares):
    """Calculate transaction fee per stock"""
    per_share_fee = shares * 0.001
    flat_fee = 2.15
    return min(per_share_fee, flat_fee)

def get_stock_data(ticker, start, end):
    """Download stock data with error handling"""
    try:
        data = ticker.history(start=start, end=end)
        return data
    except:
        return None

def calculate_technical_features(df):
    features = {}
    
    # Returns
    #features['return_5d'] = df['Close'].pct_change(5).iloc[-1]
    #features['return_10d'] = df['Close'].pct_change(10).iloc[-1]
    #features['return_20d'] = df['Close'].pct_change(20).iloc[-1]
    #features['return_60d'] = df['Close'].pct_change(60).iloc[-1]
    
    # Volatility
    features['volatility_200'] = -1*df['Close'].pct_change().rolling(200).std().iloc[-1]
    #features['volatility_60d'] = df['Close'].pct_change().rolling(60).std().iloc[-1]
    
    # Moving averages
    #features['sma_20'] = df['Close'].rolling(20).mean().iloc[-1]
    features['sma_200'] = df['Close'].rolling(200).mean().iloc[-1]
    #features['price_to_sma20'] = df['Close'].iloc[-1] / features['sma_20'] if features['sma_20'] > 0 else 0
    features['price_to_sma200'] = df['Close'].iloc[-1] / features['sma_200'] if features['sma_200'] > 0 else 0
    
    # Momentum
    #features['rsi'] = calculate_rsi(df['Close'], 14)
    features['momentum'] = df['Close'].iloc[-1] / df['Close'].iloc[-200] - 1 if len(df) > 200 else 0
    
    # Volume
    #features['volume_ratio'] = df['Volume'].iloc[-20:].mean() / df['Volume'].iloc[-60:-20].mean() if len(df) > 60 else 1
    
    return features

def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1] if not rsi.empty else 50

def check_volume_requirement(ticker, start, end):
    """Check if stock meets minimum volume requirement"""
    try:
        data = ticker.history(start=start, end=end)
        if data.empty or len(data) < 50:
            return False
        
        avg_volume = data['Volume'].mean()
        return float(avg_volume) >= 5000

    except Exception as e:
        print(f"Volume check failed for {ticker}: {e}")
        return False

def get_market_cap(ticker):
    """Get market cap in CAD"""
    try:
        info = ticker.info
        market_cap = info.get('marketCap', 0)
        
        # Convert to CAD if needed
        currency = info.get('currency', 'USD')
        if currency == 'USD':
            market_cap *= USD_TO_CAD
        
        return market_cap / 1e9  # Return in billions
    except:
        return 0

def get_sector(ticker):
    """Get stock sector"""
    try:
        return ticker.info.get('sector', 'Unknown')
    except:
        return 'Unknown'
    

# under work
def get_peg(ticker, price, end_date):
    start_date = pd.to_datetime(end_date)-pd.Timedelta(weeks=52)
    end_date = pd.to_datetime(end_date)

    net_income_df = ticker.financials.loc['Net Income']
    net_income_df.index = pd.to_datetime(net_income_df.index)
    shares = ticker.info['sharesOutstanding']

    index = 0

    for i in range (len(net_income_df)):
        date = net_income_df.index[i]
        if start_date >= date:
            index = i
            break
    
    try: 
        eps1 = net_income_df.iloc[index]/shares
        eps2 = net_income_df.iloc[index-1]/shares
        eps_growth = (eps2-eps1)/eps2*100
        pe_ratio = price/eps2

    except:
        print(f"Could not obtain net income for {ticker} from {start_date} to {end_date}")
        return "No Data"

    return pe_ratio/eps_growth

def norm (col):
    new_col = (col-col.mean())/col.std()
    return new_col








# ============================================================================
# MAIN PORTFOLIO CONSTRUCTION
# ============================================================================

# Load tickers
tickers_df = pd.read_csv('Tickers_Example.csv')

# Handle different column name formats
if 'Tickers' in tickers_df.columns:
    tickers_list = tickers_df['Tickers'].tolist()
elif 'Ticker' in tickers_df.columns:
    tickers_list = tickers_df['Ticker'].tolist()
else:
    # If no header, use first column
    tickers_list = tickers_df.iloc[:, 0].tolist()


for i in range (len(tickers_list)):
    tickers_list[i]=yf.Ticker(tickers_list[i])
    

print(f"Total tickers loaded: {len(tickers_list)}")

# Filter stocks based on volume requirement
print("\nFiltering stocks by volume requirement...")
valid_tickers = []
for ticker in tickers_list:
    if check_volume_requirement(ticker, VOLUME_CHECK_START, VOLUME_CHECK_END):
        valid_tickers.append(ticker)

print(f"Stocks passing volume filter: {len(valid_tickers)}")


# Prepare data for finding optimal weights for scoring each individual stock
col_names = ['ticker', 'momentum', 'price_to_sma200', 'volatility_200']
stock_features_dict = {'ticker': [],
                       'momentum': [], 
                       'price_to_sma200': [], 
                       'volatility_200': []}

for ticker in valid_tickers:
    print(f"Processing {ticker}...")
    data = get_stock_data(ticker, TRAINING_DAY_BEGIN, TRAINING_DAY_END)
    
    if data is None or len(data) < 200:
        print(f"No Data for {ticker}")
        continue
    
    try:
        features = calculate_technical_features(data)
        features['ticker'] = ticker

        for name in col_names:
            stock_features_dict[name].append(features[name])

    except Exception as e:
        print(f"  Error processing {ticker}: {e}")
        continue


stock_features_df = pd.DataFrame(stock_features_dict)

stock_features_df.set_index('ticker', inplace=True)
stock_features_df['momentum_norm'] = norm(stock_features_df['momentum'])
stock_features_df['price_to_sma200_norm'] = norm(stock_features_df['price_to_sma200'])
stock_features_df['volatility_200_norm'] = norm(stock_features_df['volatility_200'])


# THIS NUMBER WILL BE BIGGER WHEN WE SUBMIT
TRIALS = 200
variables = ['momentum_norm', 'price_to_sma200_norm', 'volatility_200_norm']
n = len(variables)

optimal_weights = []
max_sharpe_ratio = -100

for i in range (TRIALS):
    var_weights = np.random.rand(n)
    var_weights = var_weights/var_weights.sum()

    ticker_score = []

    for i in range (len(stock_features_df)):
        score = 0
        for j in range(n):
            score += var_weights[j]*stock_features_df[variables[j]].iloc[i]
        ticker_score.append([stock_features_df.index[i], score])

    ticker_score = sorted(ticker_score, key=lambda x: x[1], reverse=True)


    # we will figure out how to weight the stocks later
    stock_weights = []

    for i in range (MIN_STOCKS):
        stock_weights.append(1/MIN_STOCKS)


    tickers = []

    dates = first_trading_days_of_each_month(TRAINING_DAY_END, YEAR_AFTER_TRAINING_DAY_END)

    for i in range (MIN_STOCKS):
        tickers.append(ticker_score[i][0])


    df = create_df(tickers, dates, stock_weights)

    sharpe = (df['Percentage Returns'].mean()-RISK_FREE_RATE)/df['Percentage Returns'].std()

    if sharpe > max_sharpe_ratio:
        max_sharpe_ratio=sharpe
        optimal_weights= var_weights

print(f"MAXIMUM SHARPE RATIO {max_sharpe_ratio}")


# ============================================================================
# PORTFOLIO OPTIMIZATION WITH CONSTRAINTS
# ============================================================================

print("\nBuilding portfolio with constraints...")


stock_features_dict = {'ticker': [],
                       'momentum': [], 
                       'price_to_sma200': [], 
                       'volatility_200': []}

for ticker in valid_tickers:
    print(f"Processing {ticker}...")
    data = get_stock_data(ticker, datetime.now()-timedelta(weeks=53),datetime.now()-timedelta(days=7))
    
    if data is None or len(data) < 200:
        print(f"No Data for {ticker}")
        continue
    
    try:
        features = calculate_technical_features(data)
        features['ticker'] = ticker
        for name in col_names:
            stock_features_dict[name].append(features[name])

    except Exception as e:
        print(f"  Error processing {ticker}: {e}")
        continue


stock_features_df = pd.DataFrame(stock_features_dict)

stock_features_df.set_index('ticker', inplace=True)
stock_features_df['momentum_norm'] = norm(stock_features_df['momentum'])
stock_features_df['price_to_sma200_norm'] = norm(stock_features_df['price_to_sma200'])
stock_features_df['volatility_200_norm'] = norm(stock_features_df['volatility_200'])



ticker_score = []

for i in range (len(stock_features_df)):
    score = 0
    for j in range(n):
        score += optimal_weights[j]*stock_features_df[variables[j]].iloc[i]
    ticker_score.append([stock_features_df.index[i], score])



ticker_score = (sorted(ticker_score, key=lambda x: x[1], reverse= True))

for arr in ticker_score:
    print(arr)



# Select top stocks for portfolio
selected_stocks = []
sector_allocation = {}


for pair in ticker_score:
    ticker = pair[0]
    
    # Get market cap and sector
    market_cap = get_market_cap(ticker)
    sector = get_sector(ticker)

    print(f"Market Cap: {market_cap} ")
    
    if market_cap == 0:
        continue
    
    selected_stocks.append({
        'Ticker': ticker,
        'market_cap': market_cap,
        'sector': sector
    })
    
    if len(selected_stocks) >= MIN_STOCKS:
        break

# Ensure we have enough stocks
if len(selected_stocks) < MIN_STOCKS:
    print(f"Warning: Only {len(selected_stocks)} stocks available")

# Check market cap mix
has_large_cap = any(s['market_cap'] > 10 for s in selected_stocks)
has_small_cap = any(s['market_cap'] < 2 for s in selected_stocks)

print(f"\nSelected {len(selected_stocks)} stocks")
print(f"Has large-cap (>$10B): {has_large_cap}")
print(f"Has small-cap (<$2B): {has_small_cap}")

# Calculate weights (equal weight adjusted for constraints)
n_stocks = len(selected_stocks)
min_weight_per_stock = 1.0 / (2 * n_stocks)


# ignore for now
'''

mean_of_predicted_return_of_selected_stocks = portfolio_df['predicted_return'].mean()
std_of_predicted_return_of_selected_stocks = portfolio_df['predicted_return'].std()
portfolio_df['normalized_predicted_return'] = (portfolio_df['predicted_return']-mean_of_predicted_return_of_selected_stocks)/std_of_predicted_return_of_selected_stocks
portfolio_df['shifted_normalized_predict_return'] = portfolio_df['normalized_predicted_return']-portfolio_df['normalized_predicted_return'].min()

sum_shifted_normalized = portfolio_df['shifted_normalized_predict_return'].sum()
avg_shifted_normalized = sum_shifted_normalized/n_stocks
portfolio_df['Weight'] = portfolio_df['shifted_normalized_predict_return']/avg_shifted_normalized*1.0/n_stocks
'''


# Initial equal weighting
portfolio_df = pd.DataFrame(selected_stocks)
portfolio_df['Weight'] = 1.0 / n_stocks

# Adjust for max weight constraint
portfolio_df['Weight'] = portfolio_df['Weight'].clip(upper=MAX_WEIGHT)
portfolio_df['Weight'] = portfolio_df['Weight'] / portfolio_df['Weight'].sum()

# Adjust for sector constraint
sector_weights = portfolio_df.groupby('sector')['Weight'].sum()
for sector in sector_weights[sector_weights > MAX_SECTOR_WEIGHT].index:
    sector_mask = portfolio_df['sector'] == sector
    scale_factor = MAX_SECTOR_WEIGHT / sector_weights[sector]
    portfolio_df.loc[sector_mask, 'Weight'] *= scale_factor

# Renormalize
portfolio_df['Weight'] = portfolio_df['Weight'] / portfolio_df['Weight'].sum()
print("\nCalculating final portfolio...")
print(f"\nSector allocation:")
print(portfolio_df.groupby('sector')['Weight'].sum().sort_values(ascending=False))
portfolio_df['Price_USD'] = 0.0
# ============================================================================
# CALCULATE SHARES AND FINAL PORTFOLIO
# ============================================================================

print("\nCalculating final portfolio...")



# Get current prices
portfolio_df['Price_USD'] = 0.0
portfolio_df['Currency'] = ''

for idx, row in portfolio_df.iterrows():
    ticker = row['Ticker']
    current_data = get_stock_data(ticker, (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d'), datetime.now().strftime('%Y-%m-%d'))
    
    if current_data is not None and not current_data.empty:
        portfolio_df.at[idx, 'Initial_Price'] = current_data['Close'].iloc[0]
        portfolio_df.at[idx, 'Final_Price'] = current_data['Close'].iloc[-1]

        
        # Determine currency
        stock_info = ticker.info
        currency = stock_info.get('currency', 'USD')
        portfolio_df.at[idx, 'Currency'] = currency

# Calculate shares and fees
total_fees = 0
portfolio_df['Shares'] = 0.0
portfolio_df['Fee_CAD'] = 0.0


for idx, row in portfolio_df.iterrows():
    weight = row['Weight']
    price = row['Initial_Price']
    currency = row['Currency']
    
    # Convert allocation to CAD
    allocation_cad = INITIAL_CAPITAL * weight

    
    # Convert to purchase currency
    if currency == 'CAD':
        allocation_purchase = allocation_cad
        price_purchase = price
    else:  # USD
        allocation_purchase = allocation_cad / USD_TO_CAD
        price_purchase = price

    # Calculate shares (initial estimate)
    shares = allocation_purchase / price_purchase
    
    # Calculate fee
    fee_usd = calculate_fee(shares)
    fee_cad = fee_usd * USD_TO_CAD
    
    # Adjust shares to account for fees
    allocation_after_fee = allocation_purchase - fee_usd
    shares_final = allocation_after_fee / price_purchase
    
    portfolio_df.at[idx, 'Shares'] = shares_final
    portfolio_df.at[idx, 'Fee_CAD'] = fee_cad
    total_fees += fee_cad

# Calculate final values
portfolio_df['Value_CAD'] = 0.0
portfolio_df['Price'] = 0.0

for idx, row in portfolio_df.iterrows():
    shares = row['Shares']
    currency = row['Currency']
    final = row['Final_Price']
    
    if currency == 'CAD':
        value_cad = shares * final
        portfolio_df.at[idx, 'Price'] = final
    else:  # USD
        value_cad = shares * final * USD_TO_CAD
        portfolio_df.at[idx, 'Price'] = final * USD_TO_CAD
    
    portfolio_df.at[idx, 'Value_CAD'] = value_cad

total_portfolio_value = portfolio_df['Value_CAD'].sum()
portfolio_df['Weight'] = portfolio_df['Value_CAD'] / total_portfolio_value

# ============================================================================
# OUTPUT FINAL PORTFOLIO
# ============================================================================

print("\n" + "="*80)
print("FINAL PORTFOLIO")
print("="*80)

final_portfolio = portfolio_df[['Ticker', 'Price', 'Currency', 'Shares', 'Value_CAD', 'Weight']].copy()
final_portfolio.columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
final_portfolio.index = range(1, len(final_portfolio) + 1)
final_portfolio['Weight'] = (final_portfolio['Weight'] * 100).round(2)
final_portfolio['Value'] = final_portfolio['Value'].round(2)
final_portfolio['Shares'] = final_portfolio['Shares'].round(4)
final_portfolio['Price'] = final_portfolio['Price'].round(2)

print(final_portfolio)
print("\n" + "-"*80)
print(f"Total Portfolio Value: ${total_portfolio_value:,.2f} CAD")
print(f"Total Fees Paid: ${total_fees:,.2f} CAD")


print(optimal_weights)


Total tickers loaded: 40

Filtering stocks by volume requirement...


$AGN: possibly delisted; no timezone found
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


Stocks passing volume filter: 36
Processing yfinance.Ticker object <ABBV>...
Processing yfinance.Ticker object <ABT>...
Processing yfinance.Ticker object <ACN>...
Processing yfinance.Ticker object <AIG>...
Processing yfinance.Ticker object <AMZN>...
Processing yfinance.Ticker object <AXP>...
Processing yfinance.Ticker object <BA>...
Processing yfinance.Ticker object <BAC>...
Processing yfinance.Ticker object <BB.TO>...
Processing yfinance.Ticker object <BIIB>...
Processing yfinance.Ticker object <BK>...
Processing yfinance.Ticker object <BLK>...
Processing yfinance.Ticker object <BMY>...
Processing yfinance.Ticker object <C>...
Processing yfinance.Ticker object <CAT>...
Processing yfinance.Ticker object <CL>...
Processing yfinance.Ticker object <KO>...
Processing yfinance.Ticker object <LLY>...
Processing yfinance.Ticker object <LMT>...
Processing yfinance.Ticker object <MO>...
Processing yfinance.Ticker object <MRK>...
Processing yfinance.Ticker object <PEP>...
Processing yfinance.Tic